In [177]:
%load_ext autoreload

%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
import pandas as pd
import numpy as np
import os
from sentence_transformers import SentenceTransformer
import spacy
from tqdm import tqdm
from collections import defaultdict
from datasets import load_metric, load_from_disk
from differ import diff_ratio
import re

In [113]:
tqdm.pandas()

In [60]:
ls -l ../mimic_3_val.json

-rw-r--r-- 1 k1897038 users 134320361 Dec 10 17:02 ../mimic_3_val.json


In [101]:
all_docs = pd.read_json('../mimic_3_val.json', lines=True)

In [ ]:
all_docs.text = all_docs.text.apply(clean)

In [47]:
# remove excessively long documents???

In [ ]:
all_docs['text_sents'] = all_docs.text.apply(lambda t: [s.text for s in nlp(t).sents  if len(s) > 3])

In [102]:
docs = all_docs[0:10]

In [103]:
pat = re.compile(r'(?:brief)?\n?\s?hospital course:?|SUMMARY OF HOSPITAL COURSE BY SYSTEMS:?', re.IGNORECASE)
def clean(s: str):
    s = s.replace('\n\n' ,'\n')\
         .replace(r'\s{2+}', ' ')\
         .replace(r'\t', ' ')
    return pat.sub('', s)

In [104]:
docs.summary = docs.summary.apply(clean)
docs.text = docs.text.apply(clean)

/home/k1897038/anaconda3/envs/medcat/lib/python3.8/site-packages/pandas/core/generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [105]:
model = SentenceTransformer('all-MiniLM-L6-v2')

In [106]:
nlp = spacy.load('en_core_web_md')

In [107]:
docs['summ_sents'] = docs.summary.apply(lambda t: [s.text for s in nlp(t).sents if len(s) > 3])
docs['text_sents'] = docs.text.apply(lambda t: [s.text for s in nlp(t).sents  if len(s) > 3])

/tmp/ipykernel_7588/2063668814.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  docs['summ_sents'] = docs.summary.apply(lambda t: [s.text for s in nlp(t).sents if len(s) > 3])
/tmp/ipykernel_7588/2063668814.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  docs['text_sents'] = docs.text.apply(lambda t: [s.text for s in nlp(t).sents  if len(s) > 3])


In [114]:
# docs['summ_embed'] = docs.summ_sents.progress_apply(lambda sents: [model.encode(s) for s in sents])
docs['text_embed'] = docs.text_sents.progress_apply(lambda sents: [model.encode(s) for s in sents])

100%|██████████| 10/10 [01:11<00:00,  7.13s/it]
/tmp/ipykernel_7588/835971254.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  docs['text_embed'] = docs.text_sents.progress_apply(lambda sents: [model.encode(s) for s in sents])


In [17]:
docs.to_pickle('doc_embeds.pickle')

In [100]:
docs = pd.read_pickle('doc_embeds.pickle')

FileNotFoundError: [Errno 2] No such file or directory: 'doc_embeds.pickle'

In [ ]:
# train 6 seperate LSTM models for prediciting 'top-line' extractive summaries.
sent_limits = [1,2,3,5,10,15]

# LSTM embedding model
- embeddings are 'fixed' and provided by s-bert (could also be fine-tuned)
- (bi-)LSTM ranker on top of sentence embeddings (w/ or w/o attn)

In [73]:
import math

In [14]:
ds = load_from_disk('/data/users/k1897038/mimic3_dataset_pre_processed')
ds = ds.map(lambda d: {'text_embed_len': len(d['text_embed'])})
ds = ds.train_test_split(train_size=0.8, test_size=0.2, shuffle=False)
val_test_ds = ds['test'].train_test_split(train_size=0.5, test_size=0.5, shuffle=False)
train_ds = ds['train']
val_ds = val_test_ds['train']
test_ds = val_test_ds['test']

  0%|          | 0/47951 [00:00<?, ?ex/s]

In [20]:
train_ds = train_ds.sort('text_embed_len')

In [23]:
first_10 = train_ds[10:20]

In [24]:
[len(s) for s in first_10['text_embed']]

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

In [ ]:
train_ds['text_sents'][10:11]

In [35]:
len(val_ds['text_sents_limd'][0])

258

In [74]:
val_ds

Dataset({
    features: ['hadm_id', 'summary', 'text', 'clean_sum', 'clean_text', 'summ_sents', 'text_sents', 'text_embed', 'summ_lim_1', 'summ_lim_2', 'summ_lim_3', 'summ_lim_5', 'summ_lim_10', 'summ_lim_15', 'text_sents_limd', 'preds_lim_1', 'preds_lim_2', 'preds_lim_3', 'preds_lim_5', 'preds_lim_10', 'preds_lim_15', 'text_embed_len'],
    num_rows: 4795
})

In [33]:
np.array(train_ds['text_embed'][10:11]).shape

(1, 1, 384)

In [4]:
bs = 10

In [46]:
val_ds = val_ds.sort('text_embed_len')

In [49]:
len(val_ds)

4795

In [117]:
(1,2,4) + (3,)

(1, 2, 4, 3)

In [118]:
def foo():
    foo = (1,3,4)
    return foo + (1,2)

In [29]:
for i in range(math.ceil(len(val_ds) / bs)):
    in_batch = val_ds[i*bs:i*bs + bs]
    in_batch_lens = [len(s) for s in in_batch]

3836

In [ ]:
# closest matching sentences from a rouge-Lsum perspective??
# model training data is determined by number of sequenecs to extract..? 

In [19]:
metric = load_metric('rouge')
# provides 'oracle - rouge2' maximum that can be achieved by the model(s)
# for each sent in limited sent summs, find 'closest' matching extractive sentence and mark as 1, all others should be marked as 0.

In [20]:
docs.head(3)

,hadm_id,summary,text,summ_sents,text_sents,summ_embed,text_embed
0,124571,The patient was transferred to the\nIntensiv...,Radiology:CHEST (PORTABLE AP)\n 1) Possible s...,[The patient was transferred to the\nIntensive...,"[Radiology:CHEST (PORTABLE AP)\n 1) , Possibl...","[[-0.034001175, 0.009998081, -0.06825533, -0.0...","[[0.019646827, 0.09096582, -0.016166693, -0.01..."
1,161919,\nShe was admitted to ICU for close observatio...,Radiology:CT HEAD W/O CONTRAST\nKKgc MON [**21...,[\nShe was admitted to ICU for close observati...,"[Radiology:CT HEAD W/O CONTRAST\n, [**2138-5-1...","[[0.07051655, 0.03656692, 0.036904074, 0.10025...","[[0.07394422, 0.03841625, -0.025370654, -0.029..."
2,109365,The patient was taken to the operating room\...,Nursing/other:Report\nResp Care\n73 yo admitte...,[The patient was taken to the operating room\n...,[Nursing/other:Report\nResp Care\n73 yo admitt...,"[[0.042849362, 0.1184255, -0.07812561, -0.0100...","[[-0.012794435, 0.050156347, -0.011502372, -0...."


In [ ]:
sent_limits = [1,2,3,5,10,15]

In [83]:
sent_limd_sums = defaultdict(list)
for lim in sent_limits:
    docs[f'summ_lim_{lim}'] = docs.summ_sents.apply(lambda sents: sents[:lim])

/tmp/ipykernel_31791/2000559889.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  docs[f'summ_lim_{lim}'] = docs.summ_sents.apply(lambda sents: sents[:lim])


In [32]:
first_sent = docs.summ_lim_1

In [84]:
def _parse_score(lvl, scores):
    return (lvl, scores[lvl].mid.precision, scores[lvl].mid.recall, scores[lvl].mid.fmeasure)

In [85]:
for lim in sent_limits:
    col = f'summ_lim_{lim}'
    preds_srs = []
    for summ_sents, text_sents in zip(docs[col], docs.text_sents):
        sim_text_sent_idxs = []
        sents_to_compare = text_sents
        for summ_sent in summ_sents:
            sents_to_compare = [s if i not in sim_text_sent_idxs else ''
                                for i, s in enumerate(text_sents)]
            ratios = [diff_ratio(summ_sent, sent)[0] for sent in sents_to_compare]
            max_ratio_sents = np.where(ratios == np.amax(ratios))[0]
            sim_text_sent_idxs.extend(max_ratio_sents)
        preds = np.zeros(len(text_sents))
        for i in sim_text_sent_idxs:
            preds[i] = 1
        preds_srs.append(preds)
    docs[f'preds_lim_{lim}'] = preds_srs

/tmp/ipykernel_31791/3942107128.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  docs[f'preds_lim_{lim}'] = preds_srs


In [24]:
docs = docs.drop(['summary', 'text', 'summ_embed'], axis=1)

In [ ]:
docs.to_pickle('doc_embeds.pickle')

In [ ]:
# compute max ROUGE scores from 'oracle' model

In [25]:
def _parse_score(lvl, scores):
    return (lvl, scores[lvl].mid.precision, scores[lvl].mid.recall, scores[lvl].mid.fmeasure)

In [28]:
extractive_score_ceil = {}
for lim in sent_limits:
    text_sums = []
    for sent_idxs, text_sents in zip([np.argwhere(p == 1) for p in docs[f'preds_lim_{lim}']], 
                                     docs.text_sents):
        text_sum = []
        for i in sent_idxs:
            t = text_sents[i[0]]
            if t not in text_sum:
                text_sum.append(t)
        text_sums.append(''.join(text_sum))
    metric.add_batch(predictions=text_sums, references=docs[f'summ_lim_{lim}'].str.join('').tolist())
    scores = metric.compute()
    extractive_score_ceil[lim] = _parse_score('rouge1', scores), _parse_score('rouge2', scores), _parse_score('rougeLsum', scores)

In [ ]:
extractive_score_ceil

In [6]:
import torch
import torch.nn as nn
from torch.nn import LSTM, Transformer
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

In [110]:
torch.manual_seed(1)

In [112]:
docs.columns

Index(['hadm_id', 'summary', 'text', 'summ_sents', 'text_sents'], dtype='object')

In [111]:
inputs = sorted([torch.tensor(t) for t in docs.text_embed], key=lambda t: t.shape[0], reverse=True)
outputs = [torch.tensor(d_preds) for d_preds in docs.preds_lim_15]

AttributeError: 'DataFrame' object has no attribute 'text_embed'

In [87]:
in_lens = [len(i) for i in inputs]

In [88]:
outputs = torch.nn.utils.rnn.pad_sequence(outputs, batch_first=True)

In [89]:
inputs = torch.nn.utils.rnn.pad_sequence(inputs, batch_first=True)

In [62]:
class LSTMClf(nn.Module):
    def __init__(self):
        super(LSTMClf, self).__init__()
        self.model = LSTM(384, 50, num_layers=2, batch_first=True, dropout=0.5)
        self.fc = nn.Linear(50, 1)
    
    def forward(self, X, X_lens):
        X = torch.nn.utils.rnn.pack_padded_sequence(X, X_lens, batch_first=True)
        X, h = self.model(X)
        X, _ = torch.nn.utils.rnn.pad_packed_sequence(X, batch_first=True)
        X = self.fc(X)
        return X

In [63]:
model = LSTMClf()

In [64]:
criterion = nn.BCEWithLogitsLoss()
optim = torch.optim.Adam(model.parameters(), lr=0.1)

In [65]:
model.train()

LSTMClf(
  (model): LSTM(384, 50, num_layers=2, batch_first=True, dropout=0.5)
  (fc): Linear(in_features=50, out_features=1, bias=True)
)

In [66]:
outputs.shape

torch.Size([10, 4540])

In [67]:
running_loss = []
for epoch in tqdm(range(10)):
    logits = model(inputs, in_lens)
    loss = criterion(logits.squeeze(), outputs)
    loss.backward()
    running_loss.append(loss.item())
    parameters = filter(lambda p: p.requires_grad, model.parameters())
    torch.nn.utils.clip_grad_norm_(parameters, 0.25)
    optim.step()
    optim.zero_grad()

100%|██████████| 10/10 [06:45<00:00, 40.58s/it]


In [178]:
from lstm_model import MimicHospCourseDataset, pad_train_sequence, MimicHospCourseTrainDataset
from torch.utils.data import DataLoader
import torch

In [ ]:
Transformer().paramters 

In [127]:
device = "cuda"

In [128]:
ds = MimicHospCourseTrainDataset(3, size_lim=20)

Loading cached split indices for dataset at /data/users/k1897038/mimic3_dataset_pre_processed/cache-897443d7d2461d2a.arrow and /data/users/k1897038/mimic3_dataset_pre_processed/cache-78266179c3ef0613.arrow
Loading cached sorted indices for dataset at /data/users/k1897038/mimic3_dataset_pre_processed/cache-245bcf82c6035ed0.arrow


In [6]:
dl = DataLoader(ds, batch_size=10, shuffle=False, collate_fn=pad_sequence, pin_memory=True)

In [7]:
len(ds.ds['hadm_id'])

20

In [132]:
30 // 4

7

In [8]:
batches = [b for b in dl]

In [11]:
first_batch = batches[0]

In [12]:
val_inputs, val_input_lens, input_sents, _, ref_sum_sents = first_batch

In [21]:
# first example of mini-batch
len(input_sents[0])

20

In [175]:
ds = load_from_disk('/data/users/k1897038/mimic3_dataset_pre_processed/') 

In [203]:
train_ds

In [179]:
train_ds = MimicHospCourseTrainDataset(3)

Loading cached split indices for dataset at /data/users/k1897038/mimic3_dataset_pre_processed/cache-897443d7d2461d2a.arrow and /data/users/k1897038/mimic3_dataset_pre_processed/cache-78266179c3ef0613.arrow
Loading cached sorted indices for dataset at /data/users/k1897038/mimic3_dataset_pre_processed/cache-245bcf82c6035ed0.arrow


In [10]:
ds = ds.sort('text_embed_len', reverse=True)

Loading cached sorted indices for dataset at /data/users/k1897038/mimic3_dataset_pre_processed/cache-15e1a6a432eff442.arrow


In [ ]:
first_item = ds['text_embed_limd'][0]

In [19]:
embs = ds.select([0, 1, 2, 3, 4])['text_embed_limd']

In [52]:
batch = ds.select([0])

In [55]:
def trf(d):
    pass
ds.set_format(trf)

Dataset({
    features: ['hadm_id', 'summary', 'text', 'clean_sum', 'clean_text', 'summ_sents', 'text_sents', 'text_embed', 'summ_lim_1', 'summ_lim_2', 'summ_lim_3', 'summ_lim_5', 'summ_lim_10', 'summ_lim_15', 'text_sents_limd', 'preds_lim_1', 'preds_lim_2', 'preds_lim_3', 'preds_lim_5', 'preds_lim_10', 'preds_lim_15', 'text_embed_limd', 'text_embed_len'],
    num_rows: 1
})

In [62]:
ds = ds.train_test_split(train_size=0.8, test_size=0.2, shuffle=False)

In [64]:
ds = ds['train'].sort('text_embed_len', reverse=True)

In [81]:
ds.set_format('torch', columns=['text_embed_limd', 'text_embed_len', f'preds_lim_3'])

In [96]:
first_row = ds[0]

In [87]:
emb = first_row['text_embed_limd']

In [95]:
type(ds[0]['text_embed_len'])

torch.Tensor

In [8]:
from lstm_model import LSTMClf

In [3]:
model = LSTMClf()

In [5]:
sum(p.numel() for p in model.parameters() if p.requires_grad)

630601

In [9]:
sum(p.numel() for p in Transformer().parameters() if p.requires_grad)

44140544

In [139]:
model.load_state_dict(torch.load('sl3bs100/model_checkpoints/checkpoint--epoch_0-steps_200.pt'))

<All keys matched successfully>

In [143]:
optim = torch.optim.Adam(model.parameters(), lr=0.01)

In [ ]:
epoch

In [144]:
optim.state_dict()

{'state': {},
 'param_groups': [{'lr': 0.01,
   'betas': (0.9, 0.999),
   'eps': 1e-08,
   'weight_decay': 0,
   'amsgrad': False,
   'params': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17]}]}

In [147]:
torch.save({
    'model_state_dict': model.state_dict(),
    'optim_state_dict': optim.state_dict(),
    'epoch': 0,
    'batch_skip': 200,
}, 'sl3bs100/model_checkpoints/checkpoint--epoch_0-steps_200.pt')

In [ ]:
model_path = 'sl3ds50/model_checkpoints/checkpoint--epoch_9-steps_600.pt'
model.load_state_dict(torch.load(model_path))

In [160]:
torch.save({
    'model_state_dict': model.state_dict(),
    'optim_state_dict': optim.state_dict(),
    'epoch': 10,
    'batch_skip':0 
}, 'sl3ds50/model_checkpoints/checkpoint--epoch_9-steps_600.pt')

In [166]:
model_path = 'sl5ds50/model_checkpoints/checkpoint--epoch_4-steps_600.pt'
chkpt = torch.load(model_path)

In [164]:
torch.save({
    'model_state_dict': model.state_dict(),
    'optim_state_dict': optim.state_dict(),
    'epoch': 5,
    'batch_skip': 600
}, model_path)